# Import libraries and functions

In [135]:
import plotly.express as px

from functions import (
    load_data,
    group_and_aggregate_data,
    remove_sparse_columns,
    dimensionality_reduction
)

# Part 1: Compare Cities

1) Load the dataset

In [136]:
filepath = 'data/knesset_25.xlsx'  # or .csv if preferred
df = load_data(filepath)
print("Data Loaded. Shape:", df.shape)

Data Loaded. Shape: (12545, 42)


2) Group by city_name and aggregate by sum

In [137]:
city_agg_df = group_and_aggregate_data(df, group_by_column='city_name', agg_func='sum')
print("Aggregated by city_name (sum). Shape:", city_agg_df.shape)
display(city_agg_df.head())

Aggregated by city_name (sum). Shape: (1216, 42)


,city_name,ballot_code,party_avoda,party_shahar_kalkali_hadash,party_bayit_yehudi,party_agudat_israel,party_daled,party_vavmem,party_shahar_koach_hevrati,party_kama,...,party_tze'irim_bo'arim,party_manhigut_hevratit,party_kol_hasviva_vehachai,party_halev_hayehudi,party_seder_chadash,party_kol,party_beometz_bishvilech,party_kavod_umasoret,party_shas,party_daat_tov_vera
0,אבו גווייעד שבט,13.0,1,0,0,0,4,38,0,0,...,1,0,0,0,0,0,0,0,4,3
1,אבו גוש,38.2,14,1,1,3,1263,312,0,0,...,2,7,1,0,1,1,3,0,4,0
2,אבו סנאן,170.2,34,0,3,0,677,2030,4,1,...,1,4,1,3,1,6,9,0,12,1
3,אבו עבדון שבט,1.0,0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,אבו קורינאת שבט,26.5,5,0,1,0,10,65,0,0,...,0,1,0,0,2,1,0,0,3,0


3) Remove sparse columns (e.g., parties with fewer than 1000 total votes)

In [138]:
threshold_for_parties = 1000
city_cleaned_df = remove_sparse_columns(city_agg_df, threshold_for_parties)
print("After removing sparse parties, shape =", city_cleaned_df.shape)
display(city_cleaned_df.head())

After removing sparse parties, shape = (1216, 28)


,city_name,ballot_code,party_avoda,party_shahar_kalkali_hadash,party_bayit_yehudi,party_agudat_israel,party_daled,party_vavmem,party_tet,party_israel_hofshit_demokratit,...,party_raam,party_yesh_atid,party_pesofit,party_tze'irim_bo'arim,party_kol_hasviva_vehachai,party_seder_chadash,party_kol,party_beometz_bishvilech,party_shas,party_daat_tov_vera
0,אבו גווייעד שבט,13.0,1,0,0,0,4,38,0,0,...,468,1,0,1,0,0,0,0,4,3
1,אבו גוש,38.2,14,1,1,3,1263,312,13,0,...,838,26,0,2,1,1,1,3,4,0
2,אבו סנאן,170.2,34,0,3,0,677,2030,6,4,...,1160,163,0,1,1,1,6,9,12,1
3,אבו עבדון שבט,1.0,0,0,0,0,1,1,0,0,...,39,0,0,0,0,0,0,0,0,0
4,אבו קורינאת שבט,26.5,5,0,1,0,10,65,0,0,...,1096,4,1,0,0,2,1,0,3,0


4) Perform PCA, keeping city_name as meta_columns

In [139]:
meta_cols = ['city_name']
city_reduced_df = dimensionality_reduction(city_cleaned_df, num_components=2, meta_columns=meta_cols)
print("Reduced DataFrame (Cities). Shape:", city_reduced_df.shape)
display(city_reduced_df.head())

Reduced DataFrame (Cities). Shape: (1216, 3)


,city_name,PC1,PC2
0,אבו גווייעד שבט,-0.627134,-0.161529
1,אבו גוש,-0.292946,-1.786730
2,אבו סנאן,0.636294,-2.708674
3,אבו עבדון שבט,-0.766639,0.176253
4,אבו קורינאת שבט,-0.469137,-0.627456


5) Visualize the 2D PCA for cities

In [140]:
fig_cities = px.scatter(
    city_reduced_df,
    x='PC1',
    y='PC2',
    hover_data=['city_name'],
    title='Cities PCA Visualization'
)
fig_cities.show()

# Part 2: Compare Parties

1) Transpose so each row represents a party and each column a city.


In [141]:
transposed_df = city_agg_df.set_index('city_name').T
print("Transposed. Now rows = parties, columns = cities. Shape:", transposed_df.shape)
display(transposed_df.head())

Transposed. Now rows = parties, columns = cities. Shape: (41, 1216)


city_name,אבו גווייעד שבט,אבו גוש,אבו סנאן,אבו עבדון שבט,אבו קורינאת שבט,אבו קרינאת יישוב,אבו רובייעה שבט,אבו רוקייק שבט,אבו תלול,אבטין,...,תלמים,תמרת,תנובות,תעוז,תפרח,תקומה,תקוע,תראבין אצאנע שבט,תראבין אצאנעישוב,תרום
ballot_code,13.0,38.2,170.2,1.0,26.5,2.3,45.1,11.6,3.0,9.3,...,1.0,4.0,1.0,1.0,3.0,1.0,10.0,1.0,1.0,1.0
party_avoda,1.0,14.0,34.0,0.0,5.0,2.0,10.0,0.0,0.0,1.0,...,0.0,120.0,18.0,16.0,1.0,3.0,25.0,0.0,1.0,5.0
party_shahar_kalkali_hadash,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3.0,3.0,1.0,3.0,4.0,2.0,18.0,0.0,0.0,4.0
party_bayit_yehudi,0.0,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,4.0,7.0,3.0,8.0,0.0,42.0,266.0,0.0,0.0,17.0
party_agudat_israel,0.0,3.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,0.0,...,7.0,1.0,4.0,3.0,670.0,1.0,13.0,0.0,0.0,12.0


2) Remove low-vote cities (columns) if total votes across parties are below a threshold


In [142]:
city_sums = transposed_df.sum(axis=0)
low_vote_cities = city_sums[city_sums < 1000].index
transposed_cleaned_df = transposed_df.drop(columns=low_vote_cities)
print("Removed low-vote cities. Shape:", transposed_cleaned_df.shape)
display(transposed_cleaned_df.head())


Removed low-vote cities. Shape: (41, 265)


city_name,אבו גוש,אבו סנאן,אבו קורינאת שבט,אבו רובייעה שבט,אבו רוקייק שבט,אבן יהודה,אבני חפץ,אום אלפחם,אום בטין,אופקים,...,שלומי,שמשית,שעב,שער שומרון,שפרעם,שתולים,תל אביב יפו,תל מונד,תל שבע,תקוע
ballot_code,38.2,170.2,26.5,45.1,11.6,125.7,8.3,2216.9,5.0,690.5,...,51.2,6.0,31.2,414.7,1242.4,5.3,335134.5,101.6,148.7,10.0
party_avoda,14.0,34.0,5.0,10.0,0.0,540.0,0.0,51.0,1.0,116.0,...,65.0,173.0,27.0,56.0,316.0,19.0,24603.0,416.0,11.0,25.0
party_shahar_kalkali_hadash,1.0,0.0,0.0,0.0,0.0,14.0,0.0,7.0,1.0,21.0,...,10.0,2.0,0.0,21.0,3.0,12.0,1149.0,36.0,3.0,18.0
party_bayit_yehudi,1.0,3.0,1.0,0.0,0.0,59.0,23.0,3.0,0.0,89.0,...,23.0,4.0,0.0,264.0,4.0,21.0,1987.0,120.0,0.0,266.0
party_agudat_israel,3.0,0.0,0.0,5.0,0.0,26.0,10.0,16.0,0.0,2445.0,...,24.0,5.0,2.0,18.0,14.0,17.0,2204.0,19.0,1.0,13.0


3) Convert the index (party names) into a column for metadata


In [143]:
transposed_cleaned_df.reset_index(inplace=True)
transposed_cleaned_df.rename(columns={'index': 'party_name'}, inplace=True)
display(transposed_cleaned_df.head())

city_name,party_name,אבו גוש,אבו סנאן,אבו קורינאת שבט,אבו רובייעה שבט,אבו רוקייק שבט,אבן יהודה,אבני חפץ,אום אלפחם,אום בטין,...,שלומי,שמשית,שעב,שער שומרון,שפרעם,שתולים,תל אביב יפו,תל מונד,תל שבע,תקוע
0,ballot_code,38.2,170.2,26.5,45.1,11.6,125.7,8.3,2216.9,5.0,...,51.2,6.0,31.2,414.7,1242.4,5.3,335134.5,101.6,148.7,10.0
1,party_avoda,14.0,34.0,5.0,10.0,0.0,540.0,0.0,51.0,1.0,...,65.0,173.0,27.0,56.0,316.0,19.0,24603.0,416.0,11.0,25.0
2,party_shahar_kalkali_hadash,1.0,0.0,0.0,0.0,0.0,14.0,0.0,7.0,1.0,...,10.0,2.0,0.0,21.0,3.0,12.0,1149.0,36.0,3.0,18.0
3,party_bayit_yehudi,1.0,3.0,1.0,0.0,0.0,59.0,23.0,3.0,0.0,...,23.0,4.0,0.0,264.0,4.0,21.0,1987.0,120.0,0.0,266.0
4,party_agudat_israel,3.0,0.0,0.0,5.0,0.0,26.0,10.0,16.0,0.0,...,24.0,5.0,2.0,18.0,14.0,17.0,2204.0,19.0,1.0,13.0


4) Perform PCA, keeping 'party_name' as meta_columns


In [144]:
meta_cols_parties = ['party_name']
parties_reduced_df = dimensionality_reduction(
    transposed_cleaned_df,
    num_components=2,
    meta_columns=meta_cols_parties
)
print("Reduced DataFrame (Parties). Shape:", parties_reduced_df.shape)
parties_reduced_df = parties_reduced_df.map(lambda x: x.real if isinstance(x, complex) else x)
display(parties_reduced_df.head())

Reduced DataFrame (Parties). Shape: (41, 3)


,party_name,PC1,PC2
0,ballot_code,9.383502,2.403139
1,party_avoda,2.060747,-1.609121
2,party_shahar_kalkali_hadash,-3.744702,-2.831357
3,party_bayit_yehudi,-0.400878,-2.752964
4,party_agudat_israel,-0.484285,-3.012451


5) Visualize the 2D PCA for parties


In [145]:
fig_parties = px.scatter(
    parties_reduced_df,
    x='PC1',
    y='PC2',
    hover_data=['party_name'],
    title='Parties PCA Visualization'
)
fig_parties.show()